In [9]:
import nibabel as nib
import numpy as np
import os

In [10]:
def split_tmap_hemispheres(tmap_path, output_dir):
    # Load the T-map NIfTI file
    img = nib.load(str(tmap_path))
    data = img.get_fdata()
    affine = img.affine
    
    # Get the voxel indices for the middle (x=0) based on the affine
    x_voxel_coords = np.arange(data.shape[0])  # Get voxel indices along x-axis
    x_world_coords = nib.affines.apply_affine(affine, np.column_stack([x_voxel_coords, np.zeros_like(x_voxel_coords), np.zeros_like(x_voxel_coords)]))[:, 0]
    
    # Find the voxel index closest to x=0 in world space
    mid_x_index = np.argmin(np.abs(x_world_coords))

    # Create left hemisphere (erase right, x > 0)
    left_data = data.copy()
    left_data[mid_x_index:, :, :] = 0  # Zero out right hemisphere
    left_img = nib.Nifti1Image(left_data, affine, img.header)
    left_output_path = os.path.join(output_dir, f'{tmap_path.stem}_left.nii.gz')
    nib.save(left_img, left_output_path)

    # Create right hemisphere (erase left, x < 0)
    right_data = data.copy()
    right_data[:mid_x_index, :, :] = 0  # Zero out left hemisphere
    right_img = nib.Nifti1Image(right_data, affine, img.header)
    right_output_path = os.path.join(output_dir, f'{tmap_path.stem}_right.nii.gz')
    nib.save(right_img, right_output_path)

    print(f"Saved left hemisphere T-map to: {left_output_path}")
    print(f"Saved right hemisphere T-map to: {right_output_path}")



In [11]:
from pathlib import Path
second_level =Path('/home/neel/Documents/SPM_results/second_level')
for tmap in second_level.glob('**/spmT_0001.nii'):
    print(tmap)
    split_tmap_hemispheres(tmap, tmap.parent)


/home/neel/Documents/SPM_results/second_level/SPM-V_II/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II/spmT_0001_left.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II/spmT_0001_right.nii.gz
/home/neel/Documents/SPM_results/second_level/SPM-V_II_bigram/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_bigram/spmT_0001_left.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_bigram/spmT_0001_right.nii.gz
/home/neel/Documents/SPM_results/second_level/SPM-A_II/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II/spmT_0001_left.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II/spmT_0001_right.nii.gz
/home/neel/Documents/SPM_results/second_level/SPM-A_II_syllables_IPA_eSpeak_ijfix2/spmT_0001.nii
Saved left hemi

In [12]:
import numpy as np
import nibabel as nib
from scipy import ndimage

def spm_extent_threshold(input_nii, output_nii, height_threshold=3.0, extent_threshold_voxels=20):
    """
    Apply extent thresholding similar to SPM12’s approach for T-maps.
    
    The function:
      1. Loads a T-map NIfTI file.
      2. Creates a binary mask of voxels exceeding a specified height threshold.
      3. Labels contiguous clusters using an 18-connected neighborhood.
      4. Filters out clusters smaller than the specified extent (in voxels).
      5. Retains the original T values for surviving clusters and sets the rest to zero.
      6. Saves the result as a new NIfTI file.
      
    Parameters:
      input_nii (str): Path to the input T-map (NIfTI file).
      output_nii (str): Path where the output thresholded NIfTI file will be saved.
      height_threshold (float): Voxel-level threshold (e.g., T > 3.0).
      extent_threshold_voxels (int): Minimum cluster size (in voxels) required to retain a cluster.
    """
    # Load the NIfTI image
    img = nib.load(input_nii)
    data = img.get_fdata()
    
    # Create a binary mask based on the height threshold
    # (Only voxels with T values above height_threshold are considered)
    binary_mask = data > height_threshold

    # Define an 18-connected neighborhood structure.
    # Create a 3x3x3 block of ones and remove the 8 corner voxels.
    structure = np.ones((3, 3, 3), dtype=int)
    corner_offsets = [(0, 0, 0), (0, 0, 2), (0, 2, 0), (0, 2, 2),
                      (2, 0, 0), (2, 0, 2), (2, 2, 0), (2, 2, 2)]
    for cx, cy, cz in corner_offsets:
        structure[cx, cy, cz] = 0

    # Label connected clusters in the binary mask
    labels, n_clusters = ndimage.label(binary_mask, structure=structure)

    # Compute the size of each cluster (background label 0 is ignored)
    cluster_sizes = np.bincount(labels.ravel())
    
    # Identify cluster labels meeting the extent threshold (exclude background: label 0)
    valid_labels = np.where(cluster_sizes >= extent_threshold_voxels)[0]
    valid_labels = valid_labels[valid_labels != 0]  # remove the background label

    # Create a mask that is True only for voxels in valid (large enough) clusters
    valid_mask = np.isin(labels, valid_labels)

    # Generate the filtered data: keep original intensities where valid, set others to 0
    filtered_data = np.zeros_like(data)
    filtered_data[valid_mask] = data[valid_mask]

    # Save the resulting image
    filtered_img = nib.Nifti1Image(filtered_data, img.affine, img.header)
    nib.save(filtered_img, output_nii)
    
    print(f"Thresholded T-map saved to: {output_nii}")



In [ ]:
from pathlib import Path
second_level =Path('/home/neel/Documents/SPM_results/second_level')
min_cluster_size = 20
for tmap in second_level.glob('**/*.nii*'):
    if tmap.name.endswith('left.nii.gz'):
        output_file = tmap.parent / 'spmT_0001_left_Ke_20.nii.gz'
        with open(tmap.parent / 'u_threshold.txt') as f:
            height_threshold = float(f.read())
        spm_extent_threshold(tmap, output_file, height_threshold=height_threshold, extent_threshold_voxels=min_cluster_size)
    if tmap.name.endswith('right.nii.gz'):
        output_file = tmap.parent / 'spmT_0001_right_Ke_20.nii.gz'
        with open(tmap.parent / 'u_threshold.txt') as f:
            height_threshold = float(f.read())
        spm_extent_threshold(tmap, output_file, height_threshold=height_threshold, extent_threshold_voxels=min_cluster_size)
    if tmap.name == 'spmT_0001.nii':
        output_file = tmap.parent / 'spmT_0001_Ke_20.nii.gz'
        with open(tmap.parent / 'u_threshold.txt') as f:
            height_threshold = float(f.read())
        spm_extent_threshold(tmap, output_file, height_threshold=height_threshold, extent_threshold_voxels=min_cluster_size)



Thresholded T-map saved to: /home/neel/Documents/SPM_results/second_level/SPM-V_II/spmT_0001_Ke_20.nii.gz
Thresholded T-map saved to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_bigram/spmT_0001_Ke_20.nii.gz
Thresholded T-map saved to: /home/neel/Documents/SPM_results/second_level/SPM-A_II/spmT_0001_Ke_20.nii.gz
Thresholded T-map saved to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_syllables_IPA_eSpeak_ijfix2/spmT_0001_Ke_20.nii.gz
